In [18]:
import pandas as pd
import automated_reason

AnalyzedHomeBuyerInfoDF = pd.read_csv('../AnalyzedHomeBuyerInfo.csv')
AcceptedHomeBuyerInfoDF = AnalyzedHomeBuyerInfoDF[AnalyzedHomeBuyerInfoDF['Approved'] != 'N']

#Make 3 bins for the AppraisedValue column in the AcceptedHomeBuyerInfoDF dataframe
bins = pd.cut(AcceptedHomeBuyerInfoDF['AppraisedValue'], 3)
#Split the DF into 3 bins
group_names = ['Low', 'Medium', 'High']
#Add a new column to the DF with the bins
AcceptedHomeBuyerInfoDF['AppraisedValueBins'] = pd.cut(AcceptedHomeBuyerInfoDF['AppraisedValue'], 3, labels=group_names)
#Print the DF 
AcceptedHomeBuyerInfoDF.head()
LowAppraisedValue = AcceptedHomeBuyerInfoDF[AcceptedHomeBuyerInfoDF['AppraisedValueBins'] == 'Low']
MediumAppraisedValue = AcceptedHomeBuyerInfoDF[AcceptedHomeBuyerInfoDF['AppraisedValueBins'] == 'Medium']
HighAppraisedValue = AcceptedHomeBuyerInfoDF[AcceptedHomeBuyerInfoDF['AppraisedValueBins'] == 'High']

print(LowAppraisedValue["AppraisedValue"].min(), LowAppraisedValue["AppraisedValue"].max())
print(MediumAppraisedValue["AppraisedValue"].min(), MediumAppraisedValue["AppraisedValue"].max())
print(HighAppraisedValue["AppraisedValue"].min(), HighAppraisedValue["AppraisedValue"].max())

class Data(BaseModel):
    CarPayment: int
    StudentLoanPayments: int
    CreditCardPayments: int
    CreditScore: int
    GrossMonthlyIncome: int
    AppraisedValue: int
    DownPayment: int

def checkIfApproved(data: Data):
    #First see if they are approved
    result = dict()
    result["Approved"] = "N"
    if data.CreditScore < 640:
        return None
    LTV = (data.AppraisedValue - data.DownPayment) / data.AppraisedValue
    result["LTV"] = LTV
    
    if LTV > 0.95:
        return result
    
    MonthlyMortageDebt = 0
    DTI = 0
    FEDTI = 0
    for i in range(0,3):
        #Assume 8% interest rate
        fixedYears = [30, 20, 15][i]
        MonthlyMortageDebt = (data.AppraisedValue - data.DownPayment) * (0.08 / 12) / (1 - (1 + 0.08 / 12)**(-fixedYears * 12))
        result["MonthlyMortageDebt"] = MonthlyMortageDebt
        #Check if their LTV is in the grey zone
        if result["LTV"] > .8 and result["LTV"] < .95:
            MonthlyMortageDebt += data.AppraisedValue * 0.1 / 12
        
        DTI = (MonthlyMortageDebt + data.CarPayment + data.StudentLoanPayments + data.CreditCardPayments) / data.GrossMonthlyIncome
        FEDTI = MonthlyMortageDebt / data.GrossMonthlyIncome
        
        #Check if they are approved
        if DTI <= 0.36 or FEDTI <= 0.28:
            result["DTI"] = DTI
            result["FEDTI"] = FEDTI
            result["Approved"] = "Y"
            result["FixedYears"] = fixedYears
        else:
            break
    
    result["DTI"] = result.get("DTI", DTI)
    result["FEDTI"] = result.get("FEDTI", FEDTI)
    return result        
        

def userInputToCompare(data: Data):
    #First see if they are approved
    extraInfo = checkIfApproved(data)
    if extraInfo["Approved"] == "Y":
        return extraInfo["FixedYears"]
    
    #At this point they are not approved
    
    #Check for their AppraisedValue bin
    binToUse = None
    if data.AppraisedValue < 320000:
        binToUse = LowAppraisedValue
    elif data.AppraisedValue >= 320000 and data.AppraisedValue < 410000:
        binToUse = MediumAppraisedValue
    else:
        binToUse = HighAppraisedValue
        
    
    
    
    

230203.0 319811.0
320023.0 409276.0
409490.0 499130.0


/tmp/ipykernel_29982/3433862612.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AcceptedHomeBuyerInfoDF['AppraisedValueBins'] = pd.cut(AcceptedHomeBuyerInfoDF['AppraisedValue'], 3, labels=group_names)


In [13]:
MediumAppraisedValue.describe()

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyDebt,DTI,FEDTI
count,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000
mean,5027.994406,8302.587413,342.752448,423.359441,317.934266,362404.903497,67666.589678,294738.313818,1611.292730,759.419580,0.813329,2695.338883,0.312020,0.178844
std,2978.191316,1159.977311,85.464878,43.231003,72.341916,25620.954669,26542.431511,33292.165028,347.336392,58.508709,0.071763,361.674763,0.045344,0.031779
min,4.000000,5165.000000,200.000000,350.000000,200.000000,320023.000000,16200.650000,229116.290000,1015.430000,640.000000,0.710000,1907.150000,0.198535,0.106520
25%,2370.000000,7481.500000,265.000000,385.000000,253.000000,339862.000000,45497.050000,267841.930000,1324.985000,712.000000,0.760000,2416.805000,0.279067,0.155667
50%,5026.000000,8523.000000,339.000000,425.000000,313.000000,361822.000000,70876.800000,293257.440000,1598.710833,768.000000,0.800000,2680.310000,0.311949,0.177628
75%,7734.500000,9221.000000,411.000000,460.000000,380.000000,384719.500000,90113.850000,316252.590000,1832.625417,809.000000,0.870000,2910.534167,0.344130,0.201273
max,9969.000000,9999.000000,499.000000,499.000000,449.000000,409276.000000,117704.910000,388381.850000,2677.185000,849.000000,0.950000,3744.214167,0.429256,0.277414


In [14]:
HighAppraisedValue.describe()

,ID,GrossMonthlyIncome,CreditCardPayment,CarPayment,StudentLoanPayments,AppraisedValue,DownPayment,LoanAmount,MonthlyMortgagePayment,CreditScore,LTV,MonthlyDebt,DTI,FEDTI
count,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000
mean,5132.421158,8590.932136,345.920160,419.481038,319.333333,451938.221557,92731.054551,359207.167006,1893.892710,766.233533,0.795190,2978.627241,0.333548,0.205438
std,2842.526751,1036.083709,85.503694,42.588709,71.922838,26524.063071,31174.932686,35080.795811,355.904047,58.660339,0.066854,365.707288,0.040465,0.029997
min,13.000000,5610.000000,200.000000,350.000000,200.000000,409490.000000,21184.150000,291238.450000,1227.870000,640.000000,0.710000,2077.260000,0.222416,0.128550
25%,2629.000000,7920.000000,274.000000,383.000000,258.000000,428273.000000,71208.150000,332584.500000,1569.860000,721.000000,0.740000,2687.470000,0.307758,0.184862
50%,5261.000000,8811.000000,341.000000,418.000000,318.000000,451662.000000,98305.680000,356546.600000,1903.020000,774.000000,0.780000,2994.377500,0.333280,0.205231
75%,7650.000000,9472.000000,417.000000,455.000000,379.000000,476586.000000,117106.830000,382022.940000,2163.270000,820.000000,0.840000,3256.660000,0.356967,0.224221
max,9970.000000,9981.000000,499.000000,499.000000,449.000000,499130.000000,143728.930000,457137.780000,2799.935833,849.000000,0.950000,3962.773333,0.427391,0.278494


In [8]:
import os
import subprocess
import time
from bs4 import BeautifulSoup

def automated_reasoning():
    
    
    # [credit_score, down_payment, house_price, cc_debt, car_debt, student_debt, fedit]
    data = ['good', 'below', 'same', 'above', 'above', 'below', 'above']

    prolog_code = f"""
        % Predicate:
        #pred credit_score(X) :: 'You have a @(X) credit score'.
        credit_score({data[0]}). % bad, good
        #pred down_payment(X) :: 'Your down payment is @(X) expectation'.
        down_payment({data[1]}). % below, met
        #pred house_price(same) :: 'your house appraisal value is average for market'.
        #pred house_price(X) :: 'your house appraisal value is @(X) than the market average'.
        house_price({data[2]}). % below, same, above
        #pred cc_debt(X) :: 'your credit card debt is @(X) than average'.
        cc_debt({data[3]}). % below, higher
        #pred car_debt(X) :: 'your car debt is @(X) than average'.
        car_debt({data[4]}). % below, higher
        #pred student_debt(X) :: 'your student debt is @(X) than average'.
        student_debt({data[5]}). % below, higher
        #pred fedti(X) :: 'your Front-end debt to income (FEDTI) ratio @(X) expectation'.
        fedti({data[6]}). % met, above
        

        #pred suggest(X) :: 'Compare to other buyers of home at this range, the best suggestion for you currently is to @(X)'.
        
        % 1. If the user has a bad credit score and higher credit card debt, suggest paying off credit card debt first.
        suggest(pay_off_credit_card_debt_first) :- credit_score(bad), cc_debt(higher).

        % 2. If the user just has a bad credit score, suggest a way to improve the credit score.
        suggest(improve_your_credit_score) :- credit_score(bad).

        % 3. If the user's down payment is below and house value is higher or mid, suggest finding a cheaper house.
        suggest(find_a_cheaper_house) :- down_payment(below), house_price(same).
        suggest(find_a_cheaper_house) :- down_payment(below), house_price(above).

        % 4. If the user's down payment is below, but house value is below, suggest getting more income for the down payment.
        suggest(get_better_income) :- down_payment(below), house_price(below).

        % 5. If fedti is above, suggest finding a cheaper house or getting more income.
        suggest(find_a_cheaper_house) :- down_payment(user,met), fedti(above), not_cheap_house(user).
        suggest(get_better_income) :- fedti(above), house_price(low).

        % 6. Suggest the users to pay off above average debt first.
        suggest(pay_off_credit_card_debt_first) :- cc_debt(higher).
        suggest(pay_off_student_debt_first) :- student_debt(higher).
        suggest(pay_off_car_debt_first) :- car_debt(higher).

        % Find the suggestion orders for the user.
        best_suggestion(user) :- suggest(improve_your_credit_score).
        best_suggestion(user) :- suggest(pay_off_credit_card_debt_first).
        
        best_suggestion(user) :- suggest(find_a_cheaper_house).
        % Suggest the users to pay off above average debt first based on interest rate estimate.
        best_suggestion(user) :- suggest(pay_off_credit_card_debt_first).
        best_suggestion(user) :- suggest(pay_off_car_debt_first).
        best_suggestion(user) :- suggest(pay_off_student_debt_first).
        % Final suggestion, get more income.
        best_suggestion(user) :- suggest(get_better_income).

        ?- best_suggestion(user).

    """
    
    
    with open('temp.pro', 'w') as file:
        file.write(prolog_code)
        

    cmd = 'scasp temp.pro --html=alice --human'
    process = subprocess.Popen(cmd, shell=True)

    time.sleep(1/1000)

    with open('alice.html', 'r') as file:
        html = file.read()
        soup = BeautifulSoup(html, 'html.parser')
        #Get the ul tag w class="tree"
        tree = soup.find('ul', attrs={'class':'tree'})
        #Remove the text until "The best suggestion for the user currently is to"
        text = tree.text
        text = text.split('\n')
        #Reduct the space
        text = [t.strip() for t in text]
        #Remove the empty string
        text = [t for t in text if t != '']
        #Remove the first one
        text = text[1:-1]
        #Combine the text
        text = ' '.join(text)
        #Change underscore to space
        text = text.replace('_', ' ')
    return text

print(automated_reasoning())

AttributeError: 'NoneType' object has no attribute 'text'

% QUERY:I would like to know if
     'best_suggestion' holds (for user).

	ANSWER:	1 (in 0.283 ms)

MODEL:
{ best_suggestion(user),  suggest(find_a_cheaper_house),  down_payment(below),  house_price(same) }
BINDINGS: ? 